In [7]:
import os
import csv
import numpy as np
import pandas as pd

from glob import glob
from tqdm import tqdm
from bs4 import BeautifulSoup
from collections import defaultdict

from evaluation_functions import evaluation_granular, evaluate


In [8]:
section_names = {
    'S1': 'adverse reactions',
    'S2': 'boxed warnings',
    'S3': 'warnings and precautions'
}

In [63]:
folder = 'data/DeepCADRME/'
size = 5
!ls $folder

deepcadrme_guess_ades.csv guess_xml_100             guess_xml_5
guess_xml_1               guess_xml_15              guess_xml_50
guess_xml_10              guess_xml_25              guess_xml_75


In [64]:
guess_labels = glob(folder+f"guess_xml_{size}/*")

extractions = list()
for label in tqdm(guess_labels):
    drug_name = label.split('/')[-1].split('.')[0]
    with open(label, 'r') as f:
        soup = BeautifulSoup(f, 'xml')
    
    mentions = defaultdict(list)
    for mention in soup.find_all('Mention'):
        if mention['type'] == 'AdverseReaction':
          section_name = mention['section']
          mention_str = mention['str'].lower()
          mentions[section_name].append(mention_str)
    
    for section_code, ades in mentions.items():
      
      extractions.append([drug_name, section_names[section_code], ', '.join(ades).replace(', ,', '')])

extractions_df = pd.DataFrame(extractions, columns=['drug_name', 'section_name', 'gpt_output'])
extractions_df.to_csv(os.path.join('results', f"deepcadrme_{str(size).zfill(3)}_test.csv"))

100%|██████████| 99/99 [00:00<00:00, 3653.25it/s]


In [65]:
# test
drug_file = 'data/test_drug_label_text.csv'
manual_file = 'data/test_drug_label_text_manual_ades.csv'

# my_max = 10000

drugs = pd.read_csv(drug_file)
manual_ades = pd.read_csv(manual_file)
set_type = drug_file.split('/')[1].split('_')[0] # assuming file follows format "train_..." or "test...."

In [66]:
outputs = dict()
outputs[f"deepcadrme_{str(size).zfill(3)}_test"] = extractions_df

In [67]:
evaluate(outputs, manual_ades, 'strict')
evaluate(outputs, manual_ades, 'lenient')

Running strict evaluation and saving results to disk.
deepcadrme_001_test


0it [00:00, ?it/s]
/Users/TatonettiN/Projects/onsides-task1/evaluation_functions.py:113: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  overall_results = results_granular.groupby(['section','ade_type'])[['tp', 'fp', 'fn']].sum(min_count = 1).reset_index()


KeyError: 'tp'